In [ ]:
#Constraints For The Given Problem

"""
Hard Constraints (10 Points Each)
• An exam will be scheduled for each course.
• A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.
• Exam will not be held on weekends.
• Each exam must be held between 9 am and 5 pm
• Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
• A teacher cannot invigilate two exams in a row.

Soft Constraints (5 Points Each)
• All students and teachers shall be given a break on Friday from 1-2.
• A student shall not give more than 1 exam consecutively.
• Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot so the faculty meetings shall be held in parts as they are now.
• If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.

Additional Constraints (Mentioned Later on, Points not known)
• 28 Students per Class
• Use binary encoding for chromosome.

"""

#Deliverables

"""
.pynb File
One Page PDF Report
"""


In [ ]:
# %%
# All Imports

import csv
import math
import numpy as np
from random import *

In [ ]:
#Hyper Parameters

population_size = 100
crossover_rate = 0.8
mutation_rate = 0.01

h_constraints_exp = 2
s_constraints_exp = 2
dispersion_exp = 4


#Generic Variables
max_students_per_class = 28

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
total_timeslots = len(days)*2
morning_starting = '9AM'
morning_ending = '12PM'
evening_starting = '2PM'
evening_ending = '5PM'

student_data = None
teacher_data = None
course_data = None
student_in_course = None
courses_to_student = None
timeslots = None

In [ ]:
#Data Extraction from Files
def read_student_data(filename):
    #Storing student data in dictionary
    #Key is 0,1,2,3 ; value is student name
    student_data = dict()
    count = 0
    with open(filename,mode = 'r') as student_datafile:
        read = csv.reader(student_datafile,delimiter = ',')
        for name in read:
            #studentData.add(name[0])
            student_data[count] = name[0]
            count += 1
    #print(studentData)
    return student_data
def read_teacher_data(filename):
    #Storing teacher data in dictionary
    #key is 0,1,2,3 ; value is teacher names
    teacher_data = dict()
    count = 0
    with open(filename,mode = 'r') as teacher_datafile:
        read = csv.reader(teacher_datafile,delimiter = ',')
        for name in read:
            if len(name) == 0:
                continue
            teacher_data[count] = name[0]
            count += 1
    #print(teacherData)
    return teacher_data
def read_course_data(filename):
    #Storing Course data
    #Key is course code, value is course name
    courses_data = dict()
    count = 0
    with open(filename,mode = 'r') as courses_datafile:
        read = csv.reader(courses_datafile,delimiter = ',')
        for name in read:
            courses_data[name[0]] = name[1]
    #print(coursesData)
    return courses_data
def read_student_in_course_data(filename, courses_data, student_data):
    #Storing student in each course
    #Key is course; value is list of studen taking course
    student_in_course_data = dict()
    for i in courses_data:
        student_in_course_data[i] = []
    count = 0
    student_value_list = list(student_data.values())
    with open(filename, mode = 'r') as student_in_course_datafile:
        read = csv.reader(student_in_course_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            student_in_course_data[name[2]].append(student_value_list.index(name[1]))
            count += 1
    return student_in_course_data
def courses_to_student_data(filename, student_data):
    #Storing courses taken by each student
    #Key is student; value is list of courses taken by student
    courses_to_student_data = dict()
    for i in student_data:
        courses_to_student_data[i] = []
    count = 0
    student_value_list = list(student_data.values())
    with open(filename, mode = 'r') as courses_to_student_datafile:
        read = csv.reader(courses_to_student_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            courses_to_student_data[student_value_list.index(name[1])].append(name[2])
            count += 1
    #print(courses_to_student_data)
    return courses_to_student_data

def create_timeslots():
    global timeslots
    global days
    global total_timeslots
    global morning_starting
    global morning_ending
    global evening_starting
    global evening_ending
    
    timeslots = dict()
    for i in range(total_timeslots):
        start_time = None
        end_time = None
        if i%2 == 0:
            start_time = morning_starting
            end_time = morning_ending
        else:
            start_time = evening_starting
            end_time = evening_ending
        timeslots[i] = [start_time, end_time, days[int(i/2)]]

In [ ]:
#Miscellaneous Functions

#Calculates the Average of Students in Each Course
def avg_students_per_course():
    global student_in_course
    sum_students = 0
    leng = 0
    for i in student_in_course.keys():
        sum_students += len(student_in_course[i])
        leng += 1
    return (sum_students / leng)

#Calculates the number of teachers required for Invigilating the given Course Code (28 Students Max Per Teacher)
def teachers_needed(course_code):
    global student_in_course
    students = student_in_course[course_code]
    num_students = len(students)
    return math.ceil(num_students/max_students_per_class)

#Calculates the Number of Students Registered in the given Course Code
def students_count(course_code):
    global student_in_course
    return len(student_in_course[course_code])
    
#Random Debugging Print
def debug_1():
    print("")
    print(f"debug_1 Routine Called")
    print("")
    print("")
    for i in range(total_timeslots):
        print(timeslots[i])
    
    
    #Verification
    print(f"Number of Students: {len(student_data.keys())}")
    print(f"Number of Teachers: {len(teacher_data.keys())}")
    print(f"Number of Courses: {len(course_data.keys())}")
    
  
    print(f"Average Students Per Course: {avg_students}")
    print(f"Classrooms Per Course: {classrooms_per_course}")
    print(f"Average Students Per Classroom: {avg_students_per_classroom}")
    
    
    print(f"Teachers required for EE229: {teachers_needed('EE229')}")
    print(f"Students in EE229: {students_count('EE229')}")
    print(f"Teachers required for CS218: {teachers_needed('CS218')}")
    print(f"Students in CS218: {students_count('CS218')}")
    print("")
    print("")
    
def debug_2():
    print("")
    print(f"debug_2 Routine Called")
    print("")
    print("")
    #Chromosome Test
    p = chromosome()
    p.randomize()
    p.print_english_course('EE229')
    p.print_english_course('CS218')
    
    s = chromosome()
    s.randomize()
    s.print_english_course('EE229')
    s.print_english_course('CS218')
    
    child = chromosome()
    child.copy_timeslot('EE229', p)
    child.copy_teachers('EE229', p)
    child.copy_timeslot('CS218', s)
    child.copy_teachers('CS218', s)
    child.print_english_course('EE229')
    child.print_english_course('CS218')
    print("")
    print("")

In [ ]:
#Implementation of Chromosome here
class chromosome:
    #Intialize courses dictionary, course is a key, and value is 2d list, 
    #first row of list contains timeslot, second contains teachers
    def __init__(self):
        self.courses = dict()
        for key in course_data.keys():
            array_timeslots = np.zeros(total_timeslots, dtype = int)
            array_teachers = np.zeros(len(teacher_data.keys()), dtype = int)
            self.courses[key] = [array_timeslots, array_teachers]#np.vstack((array_timeslots,array_teachers)).T
    
    #Printing Chromosome Details in Binary
    def print_binary(self):
        print("-----------------------------")
        print(f"Printing Chromosome Details")
        for key in course_data.keys():
            self.print_binary_course(key)
            
    #Printing Chromosome Details of all Courses in English
    def print_english(self):
        print("-----------------------------")
        print(f"Printing Chromosome Details")
        for key in course_data.keys():
            self.print_english_course(key)
            
    #Printing Details of a Course Code in Binary
    def print_binary_course(self, course_code):
        print("--------------------")
        print(f"Course Code: {course_code}")
        print(f"Timeslots: ")
        print(f"{self.courses[course_code][0]}")
        print(f"Teachers: ")
        print(f"{self.courses[course_code][1]}")
        print("")
        
    #Printing Details of a Course Code in English
    def print_english_course(self, course_code):
        print("--------------------")
        print(f"Course Code: {course_code}")
        print(f"Timeslot: ")
        index = -1
        for i in range(total_timeslots):
            if self.courses[course_code][0][i] == 1:
                index = i
                break
        if index == -1:
            print(f"None")
        else:
            print(timeslots[index])

        index = -1
        print(f"Teacher(s): ")
        for i in range(len(teacher_data)):
            if self.courses[course_code][1][i] == 1:
                index = i
                print(teacher_data[i])
        if index == -1:
            print(f"None")
        
            
    #Get element from timeslot of teacher        
    def get_value(self,course_code, array, element):
        return self.courses[course_code][array][element]
    
    #Set element of timeslot or teacher
    def set_value(self,course_code, array, element, value):
        self.courses[course_code][array][element] = value
        
    #Get Timeslot array for a course code
    def get_timeslot(self, course_code):
        return self.courses[course_code][0]
    
    #Get Teacher array for a course code
    def get_teachers(self, course_code):
        return self.courses[course_code][1]
    
    #Make Timeslot Array Zero for a Course Code
    def reset_timeslot(self, course_code):
        self.courses[course_code][0] = np.zeros(total_timeslots, dtype = int)
    
    #Make Teacher Array Zero for a Course Code
    def reset_teachers(self, course_code):
        self.courses[course_code][1] = np.zeros(len(teacher_data.keys()), dtype = int)
        
    #Copy Timeslot Array for a Course Code from Another Chromosome
    def copy_timeslot(self, course_code, dna):
        self.courses[course_code][0] = np.copy(dna.get_timeslot(course_code))
        
    #Copy Teacher Array for a Course Code from Another Chromosome
    def copy_teachers(self, course_code, dna):
        self.courses[course_code][1] = np.copy(dna.get_teachers(course_code))
    
    #Randomize Chromosome
    def randomize(self):
        for key in course_data.keys():
            
            #Assigning a Random Timeslot
            slot_index = randint(0, total_timeslots-1)
            self.get_timeslot(key)[slot_index] = 1
            
            #Assigning Random Teachers
            max_teachers = teachers_needed(key)
            count = 0
            while count < max_teachers:
                teacher_index = randint(0, len(teacher_data.keys())-1)
                while self.get_teachers(key)[teacher_index] != 0:
                    teacher_index = randint(0, len(teacher_data.keys())-1)
                
                self.get_teachers(key)[teacher_index] = 1
                count += 1
                
    #Randomize Timeslot for a Course Code
    def randomize_timeslot(self, course_code):
        self.reset_timeslot(course_code)
        #Assigning a Random Timeslot
        slot_index = randint(0, total_timeslots-1)
        self.get_timeslot(course_code)[slot_index] = 1
        
    #Randomize Teachers for a Course Code
    def randomize_teachers(self, course_code):
        self.reset_teachers(course_code)
        
        #Assigning Random Teachers
        max_teachers = teachers_needed(course_code)
        count = 0
        while count < max_teachers:
            teacher_index = randint(0, len(teacher_data.keys())-1)
            while self.get_teachers(course_code)[teacher_index] != 0:
                teacher_index = randint(0, len(teacher_data.keys())-1)

            self.get_teachers(course_code)[teacher_index] = 1
            count += 1
        

In [ ]:
#Implementation of Individual Class
class Individual:
    def __init__(self, is_random):
        self.DNA = chromosome()
        
        self.weight_dispersion = -1
        self.weight_h_constraints = -1
        self.weight_s_constraints = -1
        
        self.student_conflicts = -1
        self.teacher_conflicts_a = -1
        self.teacher_conflicts_b = -1
        
        self.fitness = 0
        
        if is_random == True:
            self.DNA.randomize()
    
    def get_DNA(self):
        return self.DNA
            
    def calculate_fitness(self):
        #Dispersion Weight
        self.weight_dispersion = 0
        w_sum1 = 0
        for i in range(total_timeslots):
            temp_count = 0
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    temp_count += 1
            w_sum1 += math.pow(temp_count, dispersion_exp)
            
        self.weight_dispersion = w_sum1
        
        #Hard Constraints Weight
        self.weight_h_constraints = 0
        w_sum2 = 0
        #Checking Students Giving more than 1 exam at same time
        temp_count = 0
        for i in range(total_timeslots):
            cur_courses = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    cur_courses.append(j)
            checked = []
            for j in cur_courses:
                for k in cur_courses:
                    if k == j or k in checked:
                        continue
                    for student in student_in_course[j]:
                        if student in student_in_course[k]:
                            temp_count += 1
                checked.append(j)
                            
        w_sum2 += math.pow(temp_count, h_constraints_exp)
        self.student_conflicts = temp_count
        #Checking if Teacher is invigilating more than 2 exams at same time
        temp_count = 0
        for i in range(total_timeslots):
            #cur_courses = []
            assigned_teachers = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    #cur_courses.append(j)
                    temp_teachers = []
                    for k in range(len(teacher_data.keys())):
                        if self.DNA.get_teachers(j)[k] == 1:
                            temp_teachers.append(k)
                    assigned_teachers.append(temp_teachers)
            for j in range(len(assigned_teachers)):
                for k in range(j+1, len(assigned_teachers)):
                    for teacher in assigned_teachers[j]:
                        if teacher in assigned_teachers[k]:
                            temp_count += 1
        w_sum2 += math.pow(temp_count, h_constraints_exp)
        self.teacher_conflicts_a = temp_count
        #Checking if Teacher us invigilating two exams in a row
        temp_count = 0
        for i in range(total_timeslots-1):
            cur_teachers = []
            next_teachers = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    for k in range(len(teacher_data.keys())):
                        if self.DNA.get_teachers(j)[k] == 1 and k not in cur_teachers:
                            cur_teachers.append(k)
                if self.DNA.get_timeslot(j)[i+1] == 1:
                    for k in range(len(teacher_data.keys())):
                        if self.DNA.get_teachers(j)[k] == 1 and k not in next_teachers:
                            next_teachers.append(k)
            for j in cur_teachers:
                if j in next_teachers:
                    temp_count += 1
        w_sum2 += math.pow(temp_count, h_constraints_exp)
        self.teacher_conflicts_b = temp_count

        
        #Finalizing Hard Constraints Weight
        self.weight_h_constraints = w_sum2
        
        #Soft Constraints Weight
        self.weight_s_constraints = 0
        
        #Fitness
        self.fitness = 1/(self.weight_dispersion+self.weight_h_constraints+self.weight_s_constraints)
        self.fitness *= 10000
        
    def print_stats(self):
        print("------------")
        print(f"Individual Stats:")
        print(f"Fitness:{round(self.fitness, 2)} D:{self.weight_dispersion} H:{round(self.weight_h_constraints, 2)} S:{round(self.weight_s_constraints, 2)}")
        print(f"Student Conflicts:{self.student_conflicts}")
        print(f"Teacher Conflicts (Same Teacher Multiple Exams):{self.teacher_conflicts_a}")
        print(f"Teacher Conflicts (Same Teacher Twice in a Row):{self.teacher_conflicts_b}")
    

In [ ]:
if __name__ == "__main__":
    global student_data
    global teacher_data
    global course_data
    global student_in_course
    global courses_to_student
    global timeslots
    
    #Initialization
    student_data = read_student_data('studentNames.csv')
    teacher_data = read_teacher_data('teachers.csv')
    course_data = read_course_data('courses.csv')
    student_in_course = read_student_in_course_data('studentCourse.csv', course_data, student_data)
    courses_to_student = courses_to_student_data('studentCourse.csv', student_data)
    
    avg_students = avg_students_per_course()
    classrooms_per_course = math.ceil(len(teacher_data.keys())/len(course_data.keys()))
    avg_students_per_classroom = math.ceil(avg_students/classrooms_per_course)
    
    create_timeslots()    


    #Testing Individual Class
    person = Individual(True)
    person.calculate_fitness()
    person.print_stats()
    person.get_DNA().print_english()
    
    